In [1]:
import pandas as pd
import numpy as np
from scipy import stats 
import os
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import datetime
import random

In [2]:
file_cust = pd.read_csv('ecommerce_data/olist_customers_dataset.csv')
file_geo = pd.read_csv('ecommerce_data/olist_geolocation_dataset.csv')
file_items = pd.read_csv('ecommerce_data/olist_order_items_dataset.csv')
file_payments = pd.read_csv('ecommerce_data/olist_order_payments_dataset.csv')
file_reviews = pd.read_csv('ecommerce_data/olist_order_reviews_dataset.csv')
file_orders = pd.read_csv('ecommerce_data/olist_orders_dataset.csv')
file_products = pd.read_csv('ecommerce_data/olist_products_dataset.csv')
file_sellers = pd.read_csv('ecommerce_data/olist_sellers_dataset.csv')
file_category = pd.read_csv('ecommerce_data/product_category_name_translation.csv')

In [3]:
file_cust = file_cust.dropna()
file_geo = file_geo.dropna()
file_items = file_items.dropna()
file_payments = file_payments.dropna()
file_reviews = file_reviews.dropna()
file_orders = file_orders.dropna()
file_products = file_products.dropna()
file_sellers = file_sellers.dropna()
file_category = file_category.dropna()

In [4]:
data = file_orders.merge(file_items, on='order_id',how='left')
data = data.merge(file_payments, on='order_id',how='outer',validate='m:m')
data = data.merge(file_reviews, on='order_id',how='outer')
data = data.merge(file_products,on='product_id',how='outer')
data = data.merge(file_cust,on='customer_id',how='outer')
data = data.merge(file_sellers, on='seller_id',how='outer')
data = data.merge(file_category, on='product_category_name',how='outer')

In [5]:
data = data.dropna()

In [6]:
data.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'product_category_name_english'],
      dtype='object')

In [7]:
#desired columns
data['order_id']
data['customer_id']
data['order_purchase_timestamp']
data['order_item_id']
data['product_id']
data['seller_id']
data['price']

11         64.99
12         29.99
40         43.98
77         49.90
80         49.90
           ...  
121896     15.90
121988     19.99
121989     18.99
121990     18.99
121999    749.00
Name: price, Length: 11743, dtype: float64

In [8]:
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
data['order_approved_at'] = pd.to_datetime(data['order_approved_at'])
data['order_delivered_carrier_date'] = pd.to_datetime(data['order_delivered_carrier_date'])
data['order_delivered_customer_date'] = pd.to_datetime(data['order_delivered_customer_date'])
data['order_estimated_delivery_date'] = pd.to_datetime(data['order_estimated_delivery_date'])
data['shipping_limit_date'] = pd.to_datetime(data['shipping_limit_date'])
data['review_creation_date'] = pd.to_datetime(data['review_creation_date'])
data['review_answer_timestamp'] = pd.to_datetime(data['review_answer_timestamp'])

In [9]:
order_dates, order_times = zip(*[(d.date(), d.time()) for d in data['order_purchase_timestamp']])
data = data.assign(order_date=order_dates, order_time=order_times)

In [10]:
unique_orders = data.drop_duplicates(subset='order_id')
unique_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,order_date,order_time
11,4382f48f29370e70ec4ef6aa3578e1c9,e5cede01799d43206f0e40fcfe683457,delivered,2018-07-29 21:59:48,2018-07-30 18:31:39,2018-07-31 06:14:00,2018-08-03 22:09:34,2018-08-10,1.0,883cea107372a7f07b5830904f74952d,...,59f5e5c0ff8a1ca7026faa8c316693eb,17900.0,dracena,SP,13482.0,limeira,SP,housewares,2018-07-29,21:59:48
12,ff8f8f9123b73422b7a732a02483d9e2,6a029307b24c4e85eb7f8b54d28f531b,delivered,2018-08-18 16:29:06,2018-08-18 16:50:16,2018-08-21 11:27:00,2018-08-24 20:28:43,2018-09-10,1.0,22c28492de598515578d252acb18e21b,...,ef3619723058855152c4c48778a0e656,39900.0,almenara,MG,13482.0,limeira,SP,housewares,2018-08-18,16:29:06
40,e4e6269506ecfa4a94ac0188f2aa387c,9aad35717d9bc6ccafae1631ea8f0564,delivered,2018-06-15 20:18:12,2018-06-15 20:41:19,2018-06-16 08:03:00,2018-06-21 19:41:56,2018-07-13,1.0,a35c14eaa384acf7aabbbec3bd76fae6,...,45d1c56383e4e977087798ada78ac371,13425.0,piracicaba,SP,88301.0,itajai,SP,housewares,2018-06-15,20:18:12
77,874eef9a84653f570656bdb0f8519151,c59929869ce404450d73c9a018c3d3a8,delivered,2018-06-02 19:04:54,2018-06-02 19:15:17,2018-06-04 14:14:00,2018-06-12 19:36:30,2018-07-12,1.0,3d3c4219b64b1a968490e7bdfa43bf1f,...,4239137bc284d7ec558ad51001956b40,4205.0,sao paulo,SP,88301.0,itajai,SP,housewares,2018-06-02,19:04:54
80,296752c5e35a99dd06b9100ab400c8eb,40fa5cb283fa23003378da712ee22419,delivered,2018-06-23 12:19:45,2018-06-25 13:17:33,2018-06-26 14:03:00,2018-07-04 00:48:49,2018-08-06,1.0,3d3c4219b64b1a968490e7bdfa43bf1f,...,d8b877fb7950380c8d5d3093e334fcb7,35164.0,ipatinga,MG,88301.0,itajai,SP,housewares,2018-06-23,12:19:45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121896,cd38eeff6b0ebffe1200329846d83987,209cd46e6fbad8005f5a20c86a4db93c,delivered,2018-08-20 08:02:20,2018-08-20 14:11:06,2018-08-20 16:11:00,2018-08-23 19:12:52,2018-08-29,1.0,c3798d484fb730f0b5c23af0d5361595,...,d42b37e7396133bc59162e1cfea39f2f,14775.0,jaborandi,SP,7152.0,guarulhos,SP,cine_photo,2018-08-20,08:02:20
121988,493ba21a937e956c5e0e4e26c9f1b7f9,408d74d06be5d5140bb933f248de1342,delivered,2018-07-15 09:56:32,2018-07-16 13:30:36,2018-07-27 07:26:00,2018-08-02 18:06:41,2018-07-26,1.0,82d7b276f49e72ffce78d10b20518808,...,8fc8e0f5ea35f5b26e2e54f23aa2b272,4851.0,sao paulo,SP,5201.0,sao paulo,SP,books_imported,2018-07-15,09:56:32
121989,4a7cf245701068d38d441791b735e4bd,1d3b24ba06f2e3fa4bfa74fd70d2310f,delivered,2018-05-07 20:36:47,2018-05-07 20:51:50,2018-05-08 15:04:00,2018-05-09 22:38:53,2018-05-17,1.0,82d7b276f49e72ffce78d10b20518808,...,8f257a3fa12c0873312984301ef06d05,18047.0,sorocaba,SP,5201.0,sao paulo,SP,books_imported,2018-05-07,20:36:47
121990,33f8dfc5a51063c31d7b12e9d43a45fc,8f7f4871646eb2fa410cc2d50b861317,delivered,2018-04-28 14:32:54,2018-04-28 14:53:18,2018-05-02 15:25:00,2018-05-03 23:51:56,2018-05-14,1.0,82d7b276f49e72ffce78d10b20518808,...,7da3054afe371affc14d26cc2cc04358,3574.0,sao paulo,SP,5201.0,sao paulo,SP,books_imported,2018-04-28,14:32:54


In [11]:
unique_orders['customer_unique_id'].value_counts()

08e5b38d7948d37fbb2a59fc5e175ab1    3
6a9e15d6fa8ce1cabf193c21aa577f64    3
9c08d6f85c7fcec4b08ead25590c0af7    3
cfa69922f9968e0e6271647abda09b09    3
c50794dfc62b62a84f72475abf38b4e3    2
                                   ..
d292ad7eff428f42ea0eec9af09a007c    1
27e67b7b4ca8b8f0ab05e8358d2a5b2c    1
bb3d1eb35f250ae48bca3045e1f6edec    1
93b52b7591ca73fc44cdac70146a1b0d    1
76d8a7cc837890edbf3b7b2d0b09064e    1
Name: customer_unique_id, Length: 9446, dtype: int64

In [12]:
df = unique_orders['customer_unique_id'].value_counts().rename_axis('unique_values').reset_index(name='counts')

In [13]:
df

,unique_values,counts
0,08e5b38d7948d37fbb2a59fc5e175ab1,3
1,6a9e15d6fa8ce1cabf193c21aa577f64,3
2,9c08d6f85c7fcec4b08ead25590c0af7,3
3,cfa69922f9968e0e6271647abda09b09,3
4,c50794dfc62b62a84f72475abf38b4e3,2
...,...,...
9441,d292ad7eff428f42ea0eec9af09a007c,1
9442,27e67b7b4ca8b8f0ab05e8358d2a5b2c,1
9443,bb3d1eb35f250ae48bca3045e1f6edec,1
9444,93b52b7591ca73fc44cdac70146a1b0d,1


In [14]:
df1 = df.loc[df['counts'] > 1]

In [15]:
df1['counts'].sum()

206

In [16]:
repeat_list = df1['unique_values'].tolist()

In [17]:
len(repeat_list)

101

In [18]:
len(data['customer_unique_id'].unique())

9446

In [19]:
data["repeat?"] = np.where(data["customer_unique_id"].isin(repeat_list), 1, 0)


In [20]:
data['repeat?'].value_counts()

0    11475
1      268
Name: repeat?, dtype: int64

In [80]:
data.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'product_category_name_english', 'order_date',
       'order_time', 'repeat?'],
      

In [87]:
data['delivery_time b/(w)'] = data['order_estimated_delivery_date'] - data['order_delivered_customer_date']

In [88]:
data['delivery_days b/(w)'] = data['delivery_time b/(w)'].apply(lambda x: x.days)

In [89]:
data['delivery_days b/(w)']

11         6
12        16
40        21
77        29
80        32
          ..
121896     5
121988    -8
121989     7
121990    10
121999    12
Name: delivery_days b/(w), Length: 11743, dtype: int64

In [92]:
first_order_df = data.sort_values(by='order_purchase_timestamp')
first_order_df.drop_duplicates(subset ="customer_unique_id",keep = 'first', inplace = True)

In [93]:
first_order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,order_date,order_time,repeat?,delivery_delta b/(w),delivery_days b/(w),delivery_time b/(w)
66391,365a8708db2878154916b372f72e17ef,6816cc08a7c9ef50ff5bde38948ea97a,delivered,2017-05-04 19:35:21,2017-05-06 02:15:56,2017-05-09 16:53:19,2017-05-15 15:48:06,2017-06-07,1.0,1612209b0b37fbe80e1e8e1b7d19ca1d,...,90230.0,porto alegre,RS,computers_accessories,2017-05-04,19:35:21,0,22 days 08:11:54,22,22 days 08:11:54
16987,c0086672990e6dae8d6be4ccb38ab854,fe141070931f6090431da3c87c9be4ff,delivered,2017-06-15 21:19:16,2017-06-15 21:30:10,2017-06-16 15:11:39,2017-06-28 10:32:39,2017-07-06,1.0,cec09725da5ed01471d9a505e7389d37,...,17209.0,jau,SP,sports_leisure,2017-06-15,21:19:16,0,7 days 13:27:21,7,7 days 13:27:21
104800,430bd310105f069ac72ef7758f40299b,815e5251945f96c68c05d5c3500ab291,delivered,2017-09-04 18:09:28,2017-09-04 18:24:00,2017-09-05 21:34:13,2017-09-12 19:02:48,2017-09-26,1.0,9cf421049bcfaf6983bf0dfb40831a03,...,30220.0,belo horizonte,MG,art,2017-09-04,18:09:28,0,13 days 04:57:12,13,13 days 04:57:12
17755,f2b05b43011210214069c513de56b6b0,6aae5a715cdba6e7d4333a0eeeead63b,delivered,2017-10-20 00:31:45,2017-10-20 01:06:09,2017-10-24 15:18:39,2017-10-28 16:47:59,2017-11-08,1.0,759f2746cbd8fc01367864eccec63d46,...,22775.0,rio de janeiro,RJ,sports_leisure,2017-10-20,00:31:45,0,10 days 07:12:01,10,10 days 07:12:01
20766,a9f5f4fb236732250fc0b959e3a4b96e,3c4c9a40e53e5d539c9d02c03629b5f2,delivered,2017-10-23 16:26:56,2017-10-23 16:46:25,2017-10-24 21:43:56,2017-10-26 20:47:57,2017-11-09,1.0,2196663031bcde078cede855ac0b5739,...,6871.0,itapecerica da serra,SP,sports_leisure,2017-10-23,16:26:56,0,13 days 03:12:03,13,13 days 03:12:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47760,c84d88553f9878bf2c7ecda2eb211ece,c4c66f47534e09a03fc7a878a9eda5ea,delivered,2018-08-29 08:25:34,2018-08-29 08:44:13,2018-08-29 20:01:00,2018-08-30 16:56:24,2018-09-03,1.0,24bc2932a12c983f8e76d828b65cf39b,...,3679.0,sao paulo,SP,health_beauty,2018-08-29,08:25:34,0,3 days 07:03:36,3,3 days 07:03:36
60369,fb393211459aac00af932cd7ab4fa2cc,54365416b7ef5599f54a6c7821d5d290,delivered,2018-08-29 09:14:11,2018-08-29 09:25:12,2018-08-29 15:48:00,2018-08-30 13:03:28,2018-09-04,1.0,b6b76b074ed0d77d0f3443b12d8adb5e,...,5849.0,sao paulo,SP,watches_gifts,2018-08-29,09:14:11,0,4 days 10:56:32,4,4 days 10:56:32
20211,d70442bc5e3cb7438da497cc6a210f80,10a79ef2783cae3d8d678e85fde235ac,delivered,2018-08-29 10:22:35,2018-08-29 10:35:16,2018-08-29 19:57:00,2018-08-30 16:03:19,2018-09-04,1.0,9a8706b8c060b16e5f0d2925f20bc35b,...,4461.0,sao paulo,SP,sports_leisure,2018-08-29,10:22:35,0,4 days 07:56:41,4,4 days 07:56:41
26410,52018484704db3661b98ce838612b507,e450a297a7bc6839ceb0cf1a2377fa02,delivered,2018-08-29 12:25:59,2018-08-29 12:35:17,2018-08-29 13:38:00,2018-08-30 22:48:27,2018-09-03,1.0,777798445efd625458a90c13f3b3e6e7,...,5125.0,sao paulo,SP,toys,2018-08-29,12:25:59,0,3 days 01:11:33,3,3 days 01:11:33
